In [1]:
%%time 
# ------------------- PART 0: Install and Imports -------------------
from IPython.display import clear_output as c
!pip install transformers datasets sentence-transformers faiss-cpu -q
!pip install --upgrade transformers
c()
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)
from torch.utils.data import Dataset
from sentence_transformers import SentenceTransformer
import faiss
from sklearn.model_selection import train_test_split
import pickle
from typing import List
import os
os.environ["WANDB_MODE"] = "disabled"

2025-05-05 16:09:24.855362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746461365.133215      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746461365.211693      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


CPU times: user 18.3 s, sys: 3.27 s, total: 21.5 s
Wall time: 2min


In [2]:
# ------------------- PART 1: Data Loading and Preprocessing -------------------
# Load the dataset
dataset = load_dataset("qiaojin/PubMedQA", "pqa_artificial", split="train")
# Convert to pandas and keep only needed columns
df = dataset.to_pandas()[["question", "context", "long_answer"]]
# Drop rows with missing values in those columns
df = df.dropna(subset=["question", "context", "long_answer"])

# Preprocess context: join paragraphs
def preprocess_context(context):
    if isinstance(context, list):
        return " ".join(str(p) for p in context)
    elif isinstance(context, dict):
        return " ".join(str(v) for v in context.values())
    else:
        return str(context)

df['processed_context'] = df['context'].apply(preprocess_context)
df['input_text'] = 'question: ' + df['question'].astype(str) + ' context: ' + df['processed_context']
df['target_text'] = df['long_answer'].astype(str)

# Save the final DataFrame to a CSV file in /kaggle/working/
df.to_csv('/kaggle/working/pubmedqa_preprocessed.csv', index=False)

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
print(f"Train samples: {len(train_df)}, Val samples: {len(val_df)}")

README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/233M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

Train samples: 190142, Val samples: 21127


In [3]:
# ------------------- PART 2: Fine-Tuning T5 -------------------

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Dataset class
class PubMedQADataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_input_length=512, max_target_length=350):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        input_enc = self.tokenizer(row['input_text'], max_length=self.max_input_length, padding='max_length', truncation=True, return_tensors='pt')
        target_enc = self.tokenizer(row['target_text'], max_length=self.max_target_length, padding='max_length', truncation=True, return_tensors='pt')
        input_ids = input_enc['input_ids'].squeeze()
        attention_mask = input_enc['attention_mask'].squeeze()
        labels = target_enc['input_ids'].squeeze()
        labels[labels == tokenizer.pad_token_id] = -100
        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

train_dataset = PubMedQADataset(train_df, tokenizer)
val_dataset = PubMedQADataset(val_df, tokenizer)

training_args = TrainingArguments(
    output_dir="/kaggle/working//results",
    eval_steps=500, 
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=3,
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True, return_tensors="pt")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/tmp/ipykernel_19/3346152530.py:46: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=Encod

Step,Training Loss
500,2.698300
1000,2.590900
1500,2.553100
2000,2.518400
2500,2.517600
3000,2.503300
3500,2.496200
4000,2.471100
4500,2.456500
5000,2.466600


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

TrainOutput(global_step=35652, training_loss=2.395564700408802, metrics={'train_runtime': 24488.4393, 'train_samples_per_second': 23.294, 'train_steps_per_second': 1.456, 'total_flos': 7.720248244646707e+16, 'train_loss': 2.395564700408802, 'epoch': 3.0})

In [4]:
# Save fine-tuned model
model_path = "/kaggle/working/t5-small-pubmedqa"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print("Fine-tuned model saved!")

Fine-tuned model saved!


In [5]:
# ------------------- PART 3: FAISS Index Creation -------------------

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
contexts = df['processed_context'].tolist()
context_embeddings = embedding_model.encode(contexts, show_progress_bar=True)
faiss.normalize_L2(context_embeddings)

dimension = context_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(context_embeddings)

# Save index and context mapping
faiss.write_index(index, "/kaggle/working/pubmedqa_faiss.index")
context_map = {i: {"context": contexts[i], "question": df['question'].iloc[i], "answer": df['long_answer'].iloc[i]} for i in range(len(contexts))}
with open('/kaggle/working/context_map.pkl', 'wb') as f:
    pickle.dump(context_map, f)

print("FAISS index and context mapping saved!")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/6603 [00:00<?, ?it/s]

FAISS index and context mapping saved!


In [6]:
# ------------------- PART 4: RAG Functions with Custom Prompt -------------------

def retrieve_context(query, top_k=3):
    query_embedding = embedding_model.encode([query])
    faiss.normalize_L2(query_embedding)
    distances, indices = index.search(query_embedding, top_k)
    return [context_map[int(idx)]["context"] for idx in indices[0]]

def generate_answer(question, contexts, prompt_template=None):
    combined_context = " ".join(contexts)
    if prompt_template is None:
        prompt_template = "question: {question} context: {context}"
    input_text = prompt_template.format(question=question, context=combined_context)
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).input_ids
    outputs = model.generate(input_ids=input_ids, max_length=150, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def rag_pipeline(question, top_k=3, prompt_template=None):
    contexts = retrieve_context(question, top_k)
    answer = generate_answer(question, contexts, prompt_template=prompt_template)
    return {
        'question': question,
        'retrieved_contexts': contexts,
        'generated_answer': answer
    }

In [7]:
# ------------------- PART 5: Example Usage -------------------

custom_prompt = (
    "You are a helpful medical assistant. Answer the following question using the context.\n"
    "Question: {question}\nContext: {context}\nAnswer:"
)

example_question = "What is the role of ILC2s in chronic rhinosinusitis?"
result = rag_pipeline(example_question, top_k=3, prompt_template=custom_prompt)

print("\n--- Question ---")
print(result['question'])

print("\n--- Answer ---")
print(result['generated_answer'])

print("\n--- Retrieved Contexts (First 200 characters) ---")
for i, ctx in enumerate(result['retrieved_contexts']):
    print(f"\nContext {i+1}: {ctx[:200]}...")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)